In [1]:
path = "/remote-home/share/yjzheng/cscdata/data"

In [2]:

from torch.utils.data import DataLoader, Dataset

class mydataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [3]:
path = "../data/rawdata/sighan/realise/data/"
import pickle
train_dataset = pickle.load(open(path + "trainall.times2_plus.pkl", "rb"))
eval_dataset = pickle.load(open(path + "test.sighan" + "15" + "_plus.pkl", "rb"))
test_dataset = pickle.load(open(path + "test.sighan" + "15" + "_plus.pkl", "rb"))

In [9]:
print(train_dataset[0])

{'id': 'wang27k-121941', 'src': '纽约早盘作为基准的低硫轻油，五月份交割价攀升一点三四美元，来到每桶二十八点二五美元，而上周五曾下挫一豪元以上。', 'tgt': '纽约早盘作为基准的低硫轻油，五月份交割价攀升一点三四美元，来到每桶二十八点二五美元，而上周五曾下挫一美元以上。', 'tokens_size': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'src_idx': [101, 5294, 5276, 3193, 4669, 868, 711, 1825, 1114, 4638, 856, 4800, 6768, 3779, 8024, 758, 3299, 819, 769, 1200, 817, 3102, 1285, 671, 4157, 676, 1724, 5401, 1039, 8024, 3341, 1168, 3680, 3446, 753, 1282, 1061, 4157, 753, 758, 5401, 1039, 8024, 5445, 677, 1453, 758, 3295, 678, 2919, 671, 6498, 1039, 809, 677, 511, 102], 'tgt_idx': [101, 5294, 5276, 3193, 4669, 868, 711, 1825, 1114, 4638, 856, 4800, 6768, 3779, 8024, 758, 3299, 819, 769, 1200, 817, 3102, 1285, 671, 4157, 676, 1724, 5401, 1039, 8024, 3341, 1168, 3680, 3446, 753, 1282, 1061, 4157, 753, 758, 5401, 1039, 8024, 5445, 677, 1453, 758, 3295, 678, 2919, 671, 5401, 1039, 809, 677, 511,

In [11]:
_min, _max = min([ min(i["tgt_idx"]) for i in train_dataset]), max( [ max(i["tgt_idx"]) for i in train_dataset])

print(_min, _max)

1 13649


In [4]:

for i, ele in enumerate(test_dataset):
    if len(ele["src_idx"]) != len(ele["seg"]):
        print(i)
        print(len(ele["src_idx"]), len(ele["seg"]))
        print(ele["src_idx"], ele["seg"])
        break

    #if ele["src_idx"].index(101) != 0 or ele["src_idx"].index(102) != len(ele["src_idx"]) - 1:
    #    print(i)
    #    print(ele["src_idx"].index(101), ele["src_idx"].index(102))
    #    print(ele["src_idx"], ele["seg"])
    #    break


In [44]:
def _get_ReaLiSe_multi_task_dataset(args, which="15"):
    """
    For its 
    """
    print("Loading ReaLiSe with Multi-Task Dataset !")
    print("Hint: The Data You loading now is the preprocessed sighan from ReaLise, ")
    #ddp_exec("os.system('date')")

    path = "../data/rawdata/sighan/realise/data/"
    import pickle
    train_dataset = pickle.load(open(path + "trainall.times2_plus.pkl", "rb"))
    eval_dataset = pickle.load(open(path + "test.sighan" + which + "_plus.pkl", "rb"))
    test_dataset = pickle.load(open(path + "test.sighan" + which + "_plus.pkl", "rb"))

    print("Hint: Using **SIGHAN" + which + "** for eval & test !")

    def trans2mydataset(features):
        new = []
        for feature in features:
            tmp = {}
            tmp["input_ids"] = feature["src_idx"][:128]
            tmp["labels"] = feature["tgt_idx"][:128]
            tmp["seg_label"] = feature["seg"][:128]
            tmp["pos_label"] = feature["pos"][:128]
            tmp["idx_label"] = feature["error"][:128]
            tmp["attention_mask"] = ([1] * len(tmp["input_ids"]))[:128]#feature["lengths"])[:128]
            new.append(tmp)
        
        return mydataset(new)

    print("Loaded successfully !")
    #ddp_exec("os.system('date')")
    return trans2mydataset(train_dataset), trans2mydataset(eval_dataset), trans2mydataset(test_dataset)

In [36]:
train_dataset, eval_dataset, test_dataset = _get_ReaLiSe_multi_task_dataset(None, which="15")

Loading ReaLiSe with Multi-Task Dataset !
Hint: The Data You loading now is the preprocessed sighan from ReaLise, 
Hint: Using **SIGHAN15** for eval & test !
Loaded successfully !


In [24]:
test_dataset[0]

{'input_ids': [101, 872, 1962, 8013, 2769, 3221, 2476, 4263, 3152, 511, 102],
 'labels': [101, 872, 1962, 8013, 2769, 3221, 2476, 4263, 3152, 511, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}